# ICL data science challenge

## EDA

In [11]:
import pandas as pd
import numpy as np
import re

In [37]:
train_path = "../input/raw/train.csv"
preprocessed_path = "../input/derived/train-preprocessed.csv"
cols = None
data_lines = []

with open(train_path, "rt") as fin:
    with open(preprocessed_path, "wt") as fout:
        for line in fin:
            fout.write(line.replace('\\"', "'"))


# with open(train_path, "r") as f:
#     for i, line in enumerate(f):
#         if i == 0:
#             cols = [re.sub('\s+', '', x.strip('"')).strip('"') for x in line.split(" ")]
#         else:
#             data_lines.append(line.split(' '))
#             #print(len(data_lines[-1]))

# print(len(cols))

In [53]:
df = pd.read_csv(preprocessed_path, sep=r'\s+', quotechar='"')
df.iloc[233]

Index                                     234
Artist                         Camila Cabello
Track               Havana (feat. Young Thug)
Album                                  Camila
danceability                            0.765
energy                                  0.523
loudness                               -4.333
speechiness                              0.03
acousticness                            0.184
instrumentalness                     0.000036
liveness                                0.132
tempo                                 104.988
length                                 217307
time_signature                              4
A                                           0
B                                           1
C                                           1
D                                           1
AnB                                         0
AnC                                         0
AnD                                         0
BnC                               

In [48]:
df.iloc[3]['C']

0

In [51]:
temp = ['AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD',
        'AnBnC', 'AnBnD', 'AnCnD', 'BnCnD', 'AnBnCnD']
for i in range(len(df['A'])):
    for c in temp:
        row = df.iloc[i]
        if row[c] == 1:
            if 'A' in c:
                if row['A'] == 0:
                    print(row['Index'], 'A')
            if 'B' in c:
                if row['B'] == 0:
                    print(row['Index'], 'B')
            if 'C' in c:
                if row['C'] == 0:
                    print(row['Index'], 'C')
            if 'D' in c:
                if row['D'] == 0:
                    print(row['Index'], 'D')

234 A
330 B
752 A
2016 B
2261 A
2419 A
3531 B
3863 B
3873 B
3873 B
4375 A
4997 B
5230 B
5466 B
6337 A
6362 A
6627 B
7372 B
7432 A


In [43]:
df.columns

Index(['Index', 'Artist', 'Track', 'Album', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'tempo', 'length', 'time_signature', 'A', 'B', 'C', 'D',
       'AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD', 'AnBnC', 'AnBnD', 'AnCnD',
       'BnCnD', 'AnBnCnD'],
      dtype='object')